In [ ]:
import pandas as pd
import xgboost as xgb
import transformers
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import tensorflow as tf
import numpy as np

import pandas as pd
import re
from transformers import CamembertTokenizer,RobertaTokenizerFast,AutoModel, AutoTokenizer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from transformers import CamembertModel,RobertaModel
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from torch.nn.utils.clip_grad import clip_grad_norm
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
import random
import faulthandler
from gensim.test.utils import common_corpus, common_dictionary
import gensim
from gensim.utils import simple_preprocess
import nltk

import gensim.corpora as corpora
import spacy

from sentence_transformers import SentenceTransformer
import torch
import torch.nn.functional as F
import pynndescent

from sklearn.linear_model import LassoCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
corpus = pd.read_csv("model_dataset.csv")

In [ ]:
# Set the seed for reproducibility
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
kl = tf.keras.losses.KLDivergence()

In [ ]:
features = ['company_code',
 'dale_chill_formula',
 'flesch_score',
 'gunnning_fog_score',
 'verb_count',
 'noun_count',
 'adjective_count',
 'len_text',
 'len_text_words',
 'len_mentions',
 'score_group',
 'prev_likes',
 'avg_comp_likes',
 'moving_avg_likes',
 'Followers',
 'hashtag_count',
 'sadness', 'joy', 'love', 'anger', 'fear', 'surprise']

In [ ]:
# separate the target variable from the rest of the features
X = corpus[features]
y = corpus['like_count']

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False) #RobertaTokenizerFast.from_pretrained('roberta-base')
encoded_corpus = tokenizer(text=corpus.lemmatized_text.tolist(),
                            add_special_tokens=True,
                            padding='max_length',
                            truncation='longest_first',
                            return_attention_mask=True)
input_ids = encoded_corpus['input_ids']
attention_mask = encoded_corpus['attention_mask']
print("SUCCESS")

In [ ]:
def filter_long_descriptions(tokenizer, descriptions, max_len):
    indices = []
    lengths = tokenizer(descriptions, padding=False, 
                     truncation=False, return_length=True)['length']
    for i in range(len(descriptions)):
        if lengths[i] <= max_len:
            indices.append(i)
    return indices
short_descriptions = filter_long_descriptions(tokenizer, 
                               corpus.lemmatized_text.tolist(), 300)
input_ids = np.array(input_ids)[short_descriptions]
attention_mask = np.array(attention_mask)[short_descriptions]
labels = y.to_numpy()[short_descriptions]
print("SUCCESS")

In [ ]:
X_combined = pd.concat([X, pd.DataFrame(input_ids)], axis=1)

In [ ]:
test_size = 0.1
seed = seed_val
train_inputs, test_inputs, train_labels, test_labels = \
            train_test_split(X_combined, labels, test_size=test_size, 
                             random_state=seed)
train_masks, test_masks, _, _ = train_test_split(attention_mask, 
                                        labels, test_size=test_size, 
                                        random_state=seed)
print("SUCCESS")

In [ ]:
like_scaler = StandardScaler()
like_scaler.fit(train_labels.reshape(-1, 1))
train_labels = like_scaler.transform(train_labels.reshape(-1, 1))
test_labels = like_scaler.transform(test_labels.reshape(-1, 1))
print("SUCCESS")

In [ ]:
faulthandler.enable()
batch_size = 256
def create_dataloaders(inputs, masks, labels, batch_size):
    input_tensor = torch.tensor(inputs)
    mask_tensor = torch.tensor(masks)
    labels_tensor = torch.tensor(labels, dtype=torch.float32)
    dataset = TensorDataset(input_tensor, mask_tensor, 
                            labels_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size, 
                            shuffle=True)
    return dataloader
train_dataloader = create_dataloaders(train_inputs, train_masks, 
                                      train_labels, batch_size)
test_dataloader = create_dataloaders(test_inputs, test_masks, 
                                     test_labels, batch_size)
print("SUCCESS")

In [ ]:
class BERTtweetreg(nn.Module):
    
    def __init__(self, drop_rate=0.2, freeze_bertweet=False):
        
        super(BERTtweetreg, self).__init__()
        D_in, D_out = 768, 1
        
        self.bertweet = \
                   AutoModel.from_pretrained("vinai/bertweet-base")
        self.regressor = nn.Sequential(
            nn.Dropout(drop_rate),
            nn.Linear(D_in, D_out))
    def forward(self, input_ids, attention_masks):
        
        outputs = self.bertweet(input_ids, attention_masks)
        class_label_output = outputs[1]
        outputs = self.regressor(class_label_output)
        return outputs
model = BERTtweetreg(drop_rate=0.2)
print("SUCCESS")

In [ ]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print("Using GPU.")
else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")
model.to(device)
print("SUCCESS")

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr=5e-5,
                  eps=1e-8)
print("SUCCESS")

In [ ]:
epochs = 10
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,       
                 num_warmup_steps=0, num_training_steps=total_steps)
print("SUCCESS")

In [ ]:
loss_function = nn.MSELoss()
print("SUCCESS")

In [ ]:
def train(model, optimizer, scheduler, loss_function, epochs,       
          train_dataloader, device, clip_value=2):
    for epoch in range(epochs):
        print(epoch)
        print("-----")
        best_loss = 1e10
        model.train()
        for step, batch in enumerate(train_dataloader): 
            print(step)  
            batch_inputs, batch_masks, batch_labels = \
                               tuple(b.to(device) for b in batch)
            model.zero_grad()
            outputs = model(batch_inputs, batch_masks)           
            loss = loss_function(outputs.squeeze(), 
                             batch_labels.squeeze())
            loss.backward()
            clip_grad_norm(model.parameters(), clip_value)
            optimizer.step()
            scheduler.step()
  
    return model
model = train(model, optimizer, scheduler, loss_function, epochs, 
              train_dataloader, device, clip_value=2)
print("SUCCESS")

In [ ]:
def evaluate(model, loss_function, test_dataloader, device):
    model.eval()
    test_loss, test_r2 = [], []
    for batch in test_dataloader:
        batch_inputs, batch_masks, batch_labels = \
                                 tuple(b.to(device) for b in batch)
        with torch.no_grad():
            outputs = model(batch_inputs, batch_masks)
        loss = loss_function(outputs, batch_labels)
        test_loss.append(loss.item())
        r2 = r2_score(outputs, batch_labels)
        test_r2.append(r2.item())
    return test_loss, test_r2
def r2_score(outputs, labels):
    labels_mean = torch.mean(labels)
    ss_tot = torch.sum((labels - labels_mean) ** 2)
    ss_res = torch.sum((labels - outputs) ** 2)
    r2 = 1 - ss_res / ss_tot
    return r2
def predict(model, dataloader, device):
  model.eval()
  output = []
  for batch in dataloader:
      batch_inputs, batch_masks, _ = \
                                tuple(b.to(device) for b in batch)
      with torch.no_grad():
          output += model(batch_inputs, 
                          batch_masks).view(1,-1).tolist()[0]
  return output

In [ ]:
test_labels = test_labels.to_numpy()
test_labels = like_scaler.transform(test_labels.reshape(-1, 1))


In [ ]:
y_pred = predict(model, test_dataloader, device)

In [ ]:
r2_bert = r2_score(torch.FloatTensor(test_labels),torch.FloatTensor(y_pred))